In [2]:
import ee
import geemap
import os
import numpy as np
import xarray as xr
from datetime import datetime, timedelta

ee.Authenticate()
ee.Initialize(project='wildfire-lifecycle')

Define Fire Case Studies (Same as previous notebook)

In [3]:
fires = {
    "Dixie": {
        "coords": [-121.3, 40.0],
        "buffer_km": 30,
        "start_date": "2021-07-01",
        "end_date": "2021-10-01",
    },
    "Camp": {
        "coords": [-121.4, 39.8],
        "buffer_km": 30,
        "start_date": "2018-11-01",
        "end_date": "2018-12-15",
    },
    "Troublesome": {
        "coords": [-106.2, 40.0],
        "buffer_km": 30,
        "start_date": "2020-10-01",
        "end_date": "2020-11-15",
    },
    "Caldor": {
        "coords": [-120.8, 38.6],
        "buffer_km": 30,
        "start_date": "2021-08-01",
        "end_date": "2021-10-01",
    },
    "Bootleg": {
        "coords": [-121.3, 42.5],
        "buffer_km": 40,
        "start_date": "2021-07-01",
        "end_date": "2021-09-01",
    },
    "Carr": {
        "coords": [-122.3, 40.6],
        "buffer_km": 30,
        "start_date": "2018-07-23",
        "end_date": "2018-09-04",
    },
    "Thomas": {
        "coords": [-119.4, 34.5],
        "buffer_km": 30,
        "start_date": "2017-12-04",
        "end_date": "2018-01-12",
    },
    "Creek": {
        "coords": [-119.9, 37.4],
        "buffer_km": 30,
        "start_date": "2020-09-04",
        "end_date": "2020-12-24",
    },
    "Woolsey": {
        "coords": [-118.7, 34.1],
        "buffer_km": 25,
        "start_date": "2018-11-08",
        "end_date": "2018-11-21",
    },
    "Glass": {
        "coords": [-122.45, 38.55],
        "buffer_km": 25,
        "start_date": "2020-09-27",
        "end_date": "2020-10-20",
    },
}

output_dir = "../data/processed"
os.makedirs(output_dir, exist_ok=True)


Helper Functions

In [4]:
def get_aoi(fire):
    point = ee.Geometry.Point(fire["coords"])
    return point.buffer(fire["buffer_km"] * 1000).bounds()

def mask_s2_clouds(image):
    """Cloud mask for Sentinel-2."""
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
        qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

def compute_indices(image):
    """Generate NDVI, EVI, NBR."""
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }).rename('EVI')
    nbr = image.normalizedDifference(['B8', 'B12']).rename('NBR')
    return image.addBands([ndvi, evi, nbr])

def compute_spi(aoi, start_date, end_date):
    """ Compute Standardized Precipitation Index (SPI) using CHIRPS data. """
    chirps = (ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
              .filterDate(start_date, end_date)
              .filterBounds(aoi))
    
    total_precip = chirps.select("precipitation").sum().rename("precip_total")
    longterm = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate("1981-01-01", "2020-12-31").select("precipitation")
    
    mean = longterm.mean().rename("mean")
    std = longterm.reduce(ee.Reducer.stdDev()).rename("std")
    
    spi = total_precip.subtract(mean).divide(std).rename("SPI")
    return spi


def export_image(image, name, aoi, scale=500):
    out_path = os.path.join(output_dir, f"{name}.tif")
    geemap.ee_export_image(
        image,
        filename=out_path,
        scale=scale,
        region=aoi,
        file_per_band=False
    )
    print(f"Exported {name} to {out_path}")

def visualize_raster(image, vis_params, aoi, name="layer"):
    """Quick visualization using geemap."""
    Map = geemap.Map()
    Map.centerObject(aoi, 9)
    Map.addLayer(image, vis_params, name)
    return Map

Visualization Check

In [ ]:
aoi = get_aoi(fires["Dixie"])
s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterDate(fires["Dixie"]["start_date"], fires["Dixie"]["end_date"])
      .filterBounds(aoi)
      .map(mask_s2_clouds)
      .median())
s2_indices = compute_indices(s2)

Map = visualize_raster(
    s2_indices.select("NDVI"),
    {"min": 0, "max": 1, "palette": ["white", "green"]},
    aoi,
    "NDVI"
)
Map

Map(center=[39.99987156352278, -121.29928808513878], controls=(WidgetControl(options=['position', 'transparent…

Loop through fires

In [6]:
for fire_name, fire_info in fires.items():
    print(f"Processing {fire_name} Fire")
    # Standardize date keys
    start_date = fire_info.get("start_date", fire_info.get("start"))
    end_date = fire_info.get("end_date", fire_info.get("end"))
    aoi = get_aoi(fire_info)

    # Sentinel-2 Composite (cloud-masked)
    s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
          .filterDate(start_date, end_date)
          .filterBounds(aoi)
          .map(mask_s2_clouds)
          .median())
    
    s2_with_indices = compute_indices(s2)

    # Drought Index (VCI from MODIS NDVI)
    vci = (ee.ImageCollection("MODIS/061/MOD13A2")
           .filterDate(start_date, end_date)
           .filterBounds(aoi)
           .select("NDVI")
           .map(lambda img: img.unitScale(0, 10000))  # scale to 0-1
           .mean()
           .rename("VCI"))

    # SPI Calculation (precipitation anomaly)
    spi = compute_spi(aoi, start_date, end_date)

    # Burn severity label (dNBR)
    prefire = s2_with_indices.select('NBR')
    postfire = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                .filterDate(
                    (datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=30)).strftime("%Y-%m-%d"),
                    (datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=120)).strftime("%Y-%m-%d"))
                .filterBounds(aoi)
                .map(mask_s2_clouds)
                .median()
                .normalizedDifference(['B8', 'B12'])
                .rename('NBR'))
    dNBR = prefire.subtract(postfire).rename('dNBR')

    # Export layers
    export_image(s2_with_indices.select(['NDVI', 'EVI', 'NBR']), f"{fire_name}_veg_indices", aoi, scale=50)
    export_image(vci, f"{fire_name}_VCI", aoi, scale=500)
    export_image(spi, f"{fire_name}_SPI", aoi, scale=500)
    export_image(dNBR, f"{fire_name}_dNBR", aoi, scale=50)

print("Preprocessing complete")

Processing Dixie Fire
Generating URL ...
Please wait ...
Data downloaded to c:\Users\caren\Code\CAPSTONE\Wildfire-Lifecycle\data\processed\Dixie_veg_indices.tif
Exported Dixie_veg_indices to ../data/processed\Dixie_veg_indices.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\caren\Code\CAPSTONE\Wildfire-Lifecycle\data\processed\Dixie_VCI.tif
Exported Dixie_VCI to ../data/processed\Dixie_VCI.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\caren\Code\CAPSTONE\Wildfire-Lifecycle\data\processed\Dixie_SPI.tif
Exported Dixie_SPI to ../data/processed\Dixie_SPI.tif
Generating URL ...
Please wait ...
An error occurred while downloading.
Exported Dixie_dNBR to ../data/processed\Dixie_dNBR.tif
Processing Camp Fire
Generating URL ...
Please wait ...
An error occurred while downloading.
Exported Camp_veg_indices to ../data/processed\Camp_veg_indices.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\caren\Code\CAPSTONE\Wildfire-Lifecycle\data\pro